In [12]:
import carla
import numpy as np
import networkx as nx


In [13]:
START_LOCATION = {'x':-110.7, 'y':-4.3, 'z':0}
END_LOCATION = {'x':-21.6, 'y':130, 'z':0.5}
waypoints_locations = {}

In [14]:
# Connect to the default carla server
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

world = client.get_world()


In [15]:
map = world.get_map()
# map_string = map.to_opendrive()
# map.save_to_disk('map.oxdr')
waypoint_tuple_list = map.get_topology()


In [16]:
def create_graph_from_topology(carla_map):
    """
    Creates a directed NetworkX graph from a CARLA map's topology, ensuring all waypoints are included.
    """
    import networkx as nx

    graph = nx.DiGraph()

    # First, add all waypoints as nodes to ensure they are included in the graph
    all_waypoints = carla_map.generate_waypoints(2.0)  # Adjust distance as per requirement
    for waypoint in all_waypoints:
        graph.add_node(waypoint.id)

    # Retrieve the topology - a list of (waypoint, next_waypoint) tuples
    topology = carla_map.get_topology()

    for waypoint, next_waypoint in topology:
        start_id = waypoint.id
        end_id = next_waypoint.id
        distance = waypoint.transform.location.distance(next_waypoint.transform.location)
        graph.add_edge(start_id, end_id, weight=distance)
        graph.add_node(waypoint.id, x=waypoint.transform.location.x, y=waypoint.transform.location.y, z=waypoint.transform.location.z)
        if waypoint.id not in waypoints_locations:
            waypoints_locations[waypoint.id] = {'x':waypoint.transform.location.x, 'y':waypoint.transform.location.y, 'z':waypoint.transform.location.z, 'transform':waypoint.transform}


    return graph

def find_closest_waypoint(map, x, y, z):
    """
    Finds the closest waypoint to the given coordinates on the map.
    """
    location = carla.Location(x, y, z)
    return map.get_waypoint(location)

def find_nearest_node(graph, x, y, z):
    """
    Finds the nearest node in the graph to the given (x, y, z) coordinates.
    
    Assumes each node in the graph has 'x', 'y', and 'z' attributes.
    """
    min_distance = np.inf
    nearest_node = None
    
    for node in graph.nodes(data=True):
        data = node[1]
        if 'x' not in data or 'y' not in data or 'z' not in data:
            continue
        node_x, node_y, node_z = data['x'], data['y'], data['z']
        distance = np.sqrt((node_x - x) ** 2 + (node_y - y) ** 2 + (node_z - z) ** 2)
        
        if distance < min_distance:
            min_distance = distance
            nearest_node = node[0]
    
    return nearest_node

def find_shortest_path(graph, start_id, end_id):
    """
    Finds the shortest path between two waypoints in the graph using Dijkstra's algorithm.
    """
    try:
        path = nx.dijkstra_path(graph, start_id, end_id, weight='weight')
        return path
    except nx.NetworkXNoPath:
        print("No path found between the specified start and end waypoints.")
        return []




In [17]:
spawn_points = world.get_map().get_spawn_points()

# Spawn a vehicle
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
transform = spawn_points[0]
# Get the closest waypoint to the spawn point
closest_waypoint = find_closest_waypoint(map, transform.location.x, transform.location.y, transform.location.z)
vehicle = world.spawn_actor(vehicle_bp, transform)


In [18]:
# print the spawn point of the vehicle info
print("Vehicle spawn point: ", transform)
# Get location of the transform
location = transform.location
start_waypoint =  map.get_waypoint(location)
print("Start waypoint: ", start_waypoint.id)

Vehicle spawn point:  Transform(Location(x=-64.644844, y=24.471010, z=0.600000), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))
Start waypoint:  11412194709627828438


In [19]:
end_location = carla.Location(x=END_LOCATION['x'], y=END_LOCATION['y'], z=END_LOCATION['z'])
end_waypoint = map.get_waypoint(end_location)
print("End waypoint: ", end_waypoint.id)

End waypoint:  18081082011362086823


In [20]:
graph = create_graph_from_topology(map)
start = find_nearest_node(graph, START_LOCATION['x'], START_LOCATION['y'], START_LOCATION['z'])
end = find_nearest_node(graph, END_LOCATION['x'], END_LOCATION['y'], END_LOCATION['z'])

# Get a list of all graph nodes
graph_nodes = list(graph.nodes)
# Check if the start and end waypoints are in the graph
if start not in graph_nodes:
    print("Start waypoint not in the graph.")
if end not in graph_nodes:
    print("End waypoint not in the graph.")

In [21]:
route = find_shortest_path(graph, start, end)

In [22]:
# for waypoint_id in route:
#     waypoint = waypoints_locations[waypoint_id]['transform']
#     location = waypoint.location
#     vehicle.set_transform(carla.Transform(location, waypoint.rotation))
#     world.tick()

: 